# [Module 1.2] 로컬 모드 및 스크립트 모드로 훈련 (SageMaker 사용)

### 본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 1. 환경 셋업
- 2. 세이지 메이크 로컬 모드 훈련
- 3. SageMaker Host Mode 로 훈련
- 4. 모델 아티펙트 저장

---

참고:

- 세이지 메이커로 파이토치 사용 
    - [Use PyTorch with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html)


- Use PyTorch with the SageMaker Python SDK
    - https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html


- Amazon SageMaker Local Mode Examples
    - TF, Pytorch, SKLean, SKLearn Processing JOb에 대한 로컬 모드 샘플
        - https://github.com/aws-samples/amazon-sagemaker-local-mode
    - Pytorch 로컬 모드
        - https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/main/pytorch_script_mode_local_training_and_serving/pytorch_script_mode_local_training_and_serving.py    



# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [10]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
### 커스텀 라이브러리
import config 

버킷 및 폴더(prefix) 설정

In [12]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

use_default_bucket = True
if use_default_bucket:
    bucket = sagemaker_session.default_bucket()
else:
    bucket = '<Type your bucket>'
    
prefix = "NCFModel"



# 2. 세이지 메이크 로컬 모드 훈련
#### 로컬의 GPU, CPU 여부로 instance_type 결정

In [13]:
import os
import subprocess


try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
    else:
        instance_type = "local"        
except:
    pass

print("Instance type = " + instance_type)

Instance type = local_gpu


## 2.1. 스크립트 모드의 코드 작성 방법
- ![script_mode_example](img/script_mode_example.png)

## 2.2.훈련 코드 확인
- 아래의 코드는 전형적인 스크립트 모드의 코드 작성 방법을 따르고 있습니다.
- 훈련 함수는 `from train_lib import train` 로서 이전 노트북의 **[세이지 메이커 없이]** 작성한 스크래치 버전에서 사용한 훈련 함수와 동일 합니다.


In [14]:
train_code = 'src/train.py'
!pygmentize {train_code}

import argparse
import os
import json
import sys
sys.path.append('./src')

from train_lib import train

if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    ##################################
    #### 세이지 메이커 프레임워크의 도커 컨테이너 환경 변수 인자
    ##################################

    parser.add_argument('--train-data-dir', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    
    parser.add_argument('--test-data-dir', type=str, default=os.environ['SM_CHANNEL_TEST'])

    
        
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])    
           
    
    ##################################
    #### 사용자 정의 커맨드 인자
    ##################################

    parser.add_argument("--lr", 
        type=float, 
        default=0.001, 
        help="learning rate")
    parser.add_argument("--dropout", 
        type=float,
        default=0.0,  
        help="dropout rate")
    parser.add_argument("--batch_size", 
        type=int, 
        default=2

## 2.3. 로컬에 있는 데이타 세트의 위치를 지정 합니다.

In [15]:
local_inputs = config.main_path
print("local_inputs: ", local_inputs)

local_inputs:  ../data/


In [16]:
local_inputs = {'train': f'file://{local_inputs}',
          'test': f'file://{local_inputs}'}

## 2.4. 로컬 모드로 훈련 실행
- 아래의 두 라인이 로컬모드로 훈련을 지시 합니다.
```python
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
```

In [17]:
hyperparameters = {'epochs': 1, 
                   'lr': 0.001,
                   'batch_size': 256,
                   'top_k' : 10,
                   'dropout' : 0.0,
                   'factor_num' : 32,
                   'num_layers' : 3,
                   'num_ng' : 4,
                   'test_num_ng' : 99,                   
                    }  

In [21]:
from sagemaker.pytorch import PyTorch
import os
import subprocess


local_estimator = PyTorch(
    entry_point="train.py",    
    source_dir='src',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
    hyperparameters= hyperparameters               
    
)
local_estimator.fit(local_inputs)

Creating e5n5jofz4w-algo-1-gbo8i ... 
Creating e5n5jofz4w-algo-1-gbo8i ... done
Attaching to e5n5jofz4w-algo-1-gbo8i
e5n5jofz4w-algo-1-gbo8i | 2022-05-27 08:35:56,797 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
e5n5jofz4w-algo-1-gbo8i | 2022-05-27 08:35:56,875 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
e5n5jofz4w-algo-1-gbo8i | 2022-05-27 08:35:56,878 sagemaker_pytorch_container.training INFO     Invoking user training script.
e5n5jofz4w-algo-1-gbo8i | 2022-05-27 08:35:57,092 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
e5n5jofz4w-algo-1-gbo8i | /opt/conda/bin/python3.6 -m pip install -r requirements.txt
e5n5jofz4w-algo-1-gbo8i | Requirement already satisfied: nvidia-ml-py3==7.352 in /opt/conda/lib/python3.6/site-packages (from -r requirements.txt (line 1)) (7.352.0)
e5n5jofz4w-algo-1-gbo8i | Collecting pandas==0.24.2
e5n5jofz4w-algo-1-gbo8i |   Downloading

# 3. SageMaker Host Mode 로 훈련
- instance_type, session 을 수정 합니다.
- 입력 데이터를 inputs로서 S3 의 경로를 제공합니다.
- wait=False 로 지정해서 async 모드로 훈련을 실행합니다. 
- 실행 경과는 아래의 cifar10_estimator.logs() 에서 확인 합니다.

## 3.1. 데이터 세트를 S3에 업로드


In [22]:
s3_data_loc = sagemaker_session.upload_data(path=config.main_path, bucket=bucket, 
                                       key_prefix=f"{prefix}/data")
print("s3_data_loc: ", s3_data_loc)

s3_data_loc:  s3://sagemaker-us-east-1-057716757052/NCFModel/data


In [23]:
! aws s3 ls {s3_data_loc} --recursive

2022-05-24 12:16:00     128039 NCFModel/data/.ipynb_checkpoints/ml-1m.test-checkpoint.rating
2022-05-24 12:16:00   20982911 NCFModel/data/.ipynb_checkpoints/ml-1m.train-checkpoint.rating
2022-05-27 09:23:19    2891424 NCFModel/data/ml-1m.test.negative
2022-05-27 09:23:19     128039 NCFModel/data/ml-1m.test.rating
2022-05-27 09:23:19   20982911 NCFModel/data/ml-1m.train.rating
2022-05-24 12:16:00      57986 NCFModel/data/ml-1m.train.rating-small
2022-05-21 07:03:58   27404899 NCFModel/data/pinterest-20.test.negative
2022-05-21 07:03:54     807267 NCFModel/data/pinterest-20.test.rating
2022-05-21 07:03:55   21138451 NCFModel/data/pinterest-20.train.rating


## 3.2. 훈련 및 테스트 데이터를 S3 로 지정

In [24]:
s3_inputs = {
            'train': f'{s3_data_loc}',
            'test': f'{s3_data_loc}'
            }

print("s3_inputs: \n", s3_inputs)

s3_inputs: 
 {'train': 's3://sagemaker-us-east-1-057716757052/NCFModel/data', 'test': 's3://sagemaker-us-east-1-057716757052/NCFModel/data'}


## 3.3. 실험 세팅

### 실험(Experiment) 세팅
- Amazon SageMaker 실험은 기계 학습 실험을 구성, 추적, 비교 및 평가할 수 있는 Amazon SageMaker 의 기능입니다
- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 실험을 통한 Machine Learning 관리](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/experiments.html)
- sagemaker experiment는 추가적인 패키지를 설치하여야 합니다. 1_Setup_Environment 가 실행이 안되었다고 하면, `!pip install --upgrade sagemaker-experiments` 를 통해 설치 해주세요.
- 여기서는 boto3 API를 통해서 실험을 생성합니다. SageMaker Python SDK를 통해서도 가능합니다.


### 실험 생성

In [25]:
# !pip install --upgrade sagemaker-experiments
import boto3
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

from datetime import datetime

sm_client = boto3.client('sagemaker')


# 설험에 대한 이름을 생성 합니다.
experiment_name = prefix + '-single-train'

# 실험이 존재하지 않으면 생성하고, 그렇지 않으면 지나갑니다.
try:
    response = sm_client.describe_experiment(ExperimentName=experiment_name)
    print(f"Experiment:{experiment_name} already exists")    
    
except:
    response = sm_client.create_experiment(
        ExperimentName = experiment_name,
        Description = 'Experiment for NCF',
    )
    print(f"Experiment:{experiment_name} is created")        


Experiment:NCFModel-single-train already exists


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


### 하이퍼 파라미터 세팅
- epochs 값을 조절해서 실행 시간을 조정 하세요.

In [26]:
host_hyperparameters = {'epochs': 5, 
                       'lr': 0.001,
                       'batch_size': 256,
                       'top_k' : 10,
                       'dropout' : 0.0,
                       'factor_num' : 32,
                       'num_layers' : 3,
                       'num_ng' : 4,
                       'test_num_ng' : 99,                   
                    }  

### 시도(Trial) 생성

In [27]:
from datetime import datetime
# 시도 이름 생성
ts = datetime.now().strftime('%Y-%m-%d-%H-%M-%S-%f')
trial_name = experiment_name + f"-{ts}"

# 1개의 실험 안에 시도를 생성함.
response = sm_client.create_trial(
    ExperimentName = experiment_name,
    TrialName = trial_name,
)    

# 실험 설정: 실험 이름, 시도 이름으로 구성
experiment_config = {
    'ExperimentName' : experiment_name,
    'TrialName' : trial_name,
    "TrialComponentDisplayName" : 'Training',
}    



## 3.4 훈련 실행


### 훈련 메트릭을 CloudWatch 에서 보기
- 개발자 가이드
    - [Monitor and Analyze Training Jobs Using Amazon CloudWatch ](https://docs.amazonaws.cn/en_us/sagemaker/latest/dg/training-metrics.html#define-train-metrics)

In [28]:
metric_definitions=[
       {'Name': 'HR', 'Regex': 'HR=(.*?);'},
       {'Name': 'NDCG', 'Regex': 'NDCG=(.*?);'},
       {'Name': 'Loss', 'Regex': 'Loss=(.*?);'}        
    ]


In [29]:
from sagemaker.pytorch import PyTorch

instance_type = 'ml.p3.2xlarge'

host_estimator = PyTorch(
    entry_point="train.py",    
    source_dir='src',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type,
    session = sagemaker.Session(), # 세이지 메이커 세션
    hyperparameters=host_hyperparameters,
    metric_definitions = metric_definitions
    
)
host_estimator.fit(s3_inputs, 
                   experiment_config = experiment_config, # 실험 설정 제공                   
                   wait=False)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-training-2022-05-27-09-23-31-515


In [30]:
%%time

host_estimator.logs()

2022-05-27 09:23:32 Starting - Starting the training job...ProfilerReport-1653643411: InProgress
...
2022-05-27 09:24:15 Starting - Preparing the instances for training......
2022-05-27 09:25:22 Downloading - Downloading input data...
2022-05-27 09:25:56 Training - Downloading the training image...........................
2022-05-27 09:30:34 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-27 09:30:37,038 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-27 09:30:37,062 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-27 09:30:37,070 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-27 09:30:37,562 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install

# 4. 실험 결과 보기


위의 실험한 결과를 확인 합니다.
- 각각의 훈련잡의 시도에 대한 훈련 사용 데이터, 모델 입력 하이퍼 파라미터, 모델 평가 지표, 모델 아티펙트 결과 위치 등의 확인이 가능합니다.
- **아래의 모든 내용은 SageMaker Studio 를 통해서 직관적으로 확인이 가능합니다.**

In [31]:
from sagemaker.analytics import ExperimentAnalytics
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 5
pd.options.display.max_colwidth = 50

search_expression = {
    "Filters": [
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}


trial_component_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
)

trial_component_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,batch_size,dropout,epochs,factor_num,log_interval,lr,num_layers,num_ng,sagemaker_container_log_level,sagemaker_job_name,sagemaker_program,sagemaker_region,sagemaker_submit_directory,test_num_ng,top_k,test - MediaType,test - Value,train - MediaType,...,BCEWithLogitsLoss_output_0_GLOBAL - Last,BCEWithLogitsLoss_output_0_GLOBAL - Count,Loss - Min,Loss - Max,Loss - Avg,Loss - StdDev,Loss - Last,Loss - Count,HR - Min,HR - Max,HR - Avg,HR - StdDev,HR - Last,HR - Count,NDCG - Min,NDCG - Max,NDCG - Avg,NDCG - StdDev,NDCG - Last,NDCG - Count,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,sagemaker_mpi_custom_mpi_options,sagemaker_mpi_enabled,sagemaker_mpi_num_of_processes_per_host
0,pytorch-training-2022-05-27-09-50-12-877-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/p...,1.0,ml.p4d.24xlarge,30.0,256.0,0.0,10.0,32.0,500.0,0.001,3.0,4.0,20.0,"""pytorch-training-2022-05-27-09-50-12-877""","""train_horovod.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/pytorch...",99.0,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pytorch-training-2022-05-27-09-23-31-515-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/p...,1.0,ml.p3.2xlarge,30.0,256.0,0.0,10.0,32.0,NaN,0.001,3.0,4.0,20.0,"""pytorch-training-2022-05-27-09-23-31-515""","""train.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/pytorch...",99.0,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,...,0.693132,1.0,0.13464,0.349566,0.221816,0.045782,0.204355,380.0,0.624,0.673,0.6572,0.014536,0.644,10.0,0.364,0.394,0.384000,0.009333,0.376,10.0,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,pytorch-training-2022-05-22-03-52-23-671-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/p...,1.0,ml.p3.2xlarge,30.0,256.0,0.0,10.0,32.0,NaN,0.001,3.0,4.0,20.0,"""pytorch-training-2022-05-22-03-52-23-671""","""train.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/pytorch...",99.0,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,...,0.252895,390.0,NaN,NaN,NaN,NaN,NaN,NaN,0.630,0.696,0.6825,0.019609,0.687,10.0,0.366,0.421,0.410200,0.017080,0.417,10.0,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...,NaN,NaN,NaN
14,pytorch-training-2022-05-22-03-16-01-173-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/p...,1.0,ml.p3.2xlarge,30.0,256.0,0.0,3.0,32.0,NaN,0.001,3.0,4.0,20.0,"""pytorch-training-2022-05-22-03-16-01-173""","""train.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/pytorch...",99.0,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,...,0.250393,117.0,NaN,NaN,NaN,NaN,NaN,NaN,0.625,0.685,0.6600,0.031225,0.685,3.0,0.358,0.405,0.384667,0.024132,0.405,3.0,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...,NaN,NaN,NaN


### 모델 평가 지표에 순서에 따른 시도 보기
- 아래는 모델 평가 지표에 따른 순서로 보여주기 입니다.

In [32]:

trial_component_training_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
    sort_by="metrics.HR.max",        
    sort_order="Descending",
    metric_names=["HR"],    
    parameter_names=["epochs", "batch_size", "dropout", "factor_num", 
                     "lr", "num_layers" ,"num_ng", "test_num_ng", "top_k",
                    ],
)

trial_component_training_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,batch_size,dropout,epochs,factor_num,lr,num_layers,num_ng,test_num_ng,top_k,HR - Min,HR - Max,HR - Avg,HR - StdDev,HR - Last,HR - Count,test - MediaType,test - Value,train - MediaType,train - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments
0,pytorch-training-2022-05-22-03-52-23-671-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,256.0,0.0,10.0,32.0,0.001,3.0,4.0,99.0,10.0,0.630,0.696,0.6825,0.019609,0.687,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...,[NCFModel-single-train-2022-05-22-03-52-16-349...,[NCFModel-single-train]
1,pytorch-training-2022-05-27-02-15-27-907-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,256.0,0.0,10.0,32.0,0.001,3.0,4.0,99.0,10.0,0.596,0.694,0.6703,0.029628,0.679,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...,[NCFModel-single-train-2022-05-27-02-15-22-682...,[NCFModel-single-train]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,pytorch-training-2022-05-26-13-36-29-023-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,256.0,0.0,10.0,32.0,0.001,3.0,4.0,99.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...,[NCFModel-single-train-2022-05-26-13-36-15-959...,[NCFModel-single-train]
14,pytorch-training-2022-05-27-09-50-12-877-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,256.0,0.0,10.0,32.0,0.001,3.0,4.0,99.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/,NaN,NaN,[NCFModel-single-train-2022-05-27-09-50-04-976...,[NCFModel-single-train]


# 5. 모델 아티펙트 저장
- S3 에 저장된 모델 아티펙트를 저장하여 추론시 사용합니다.

In [33]:
artifact_path = host_estimator.model_data
print("artifact_path: ", artifact_path)

%store artifact_path

artifact_path:  s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-05-27-09-23-31-515/output/model.tar.gz
Stored 'artifact_path' (str)


기타 변수 저장

In [34]:
%store bucket 
%store prefix

Stored 'bucket' (str)
Stored 'prefix' (str)
